In [9]:
import pandas as pd
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import plotly
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output

In [ ]:
###read data

In [6]:
data = pd.read_csv("https://firms2.modaps.eosdis.nasa.gov/data/active_fire/suomi-npp-viirs-c2/csv/SUOMI_VIIRS_C2_USA_contiguous_and_Hawaii_7d.csv")
data.head(20)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,confidence,version,bright_ti5,frp,daynight
0,46.04253,-73.14091,297.3,0.40,0.44,2020-09-27,624,N,nominal,2.0NRT,262.9,1.7,N
1,45.83951,-73.25140,306.2,0.40,0.44,2020-09-27,624,N,nominal,2.0NRT,283.9,1.5,N
2,45.83541,-73.25214,312.2,0.40,0.44,2020-09-27,624,N,nominal,2.0NRT,283.0,1.5,N
3,43.49350,-79.62478,298.6,0.39,0.59,2020-09-27,624,N,nominal,2.0NRT,285.0,0.4,N
4,43.48904,-79.62537,302.1,0.39,0.59,2020-09-27,624,N,nominal,2.0NRT,281.1,1.1,N
5,43.26948,-79.80432,295.5,0.39,0.60,2020-09-27,624,N,nominal,2.0NRT,271.3,1.6,N
6,43.27129,-79.80431,300.5,0.39,0.59,2020-09-27,624,N,nominal,2.0NRT,271.8,1.3,N
7,43.26039,-79.81023,311.4,0.39,0.60,2020-09-27,624,N,nominal,2.0NRT,268.6,0.8,N
8,43.12133,-79.17392,298.4,0.36,0.58,2020-09-27,624,N,nominal,2.0NRT,281.6,0.4,N
9,42.30130,-83.15961,297.3,0.57,0.69,2020-09-27,624,N,nominal,2.0NRT,278.7,1.7,N


In [28]:
data['timestamp']=pd.to_datetime(data['acq_date'].astype(str) + data['acq_time'].astype(str), format="%Y-%m-%d%H%M").astype(str)
 data['timestamp'].astype(str) + "\nConfidence: " + data['confidence'].astype(str) + "\nBrightness I-4: "+ data['bright_ti4'].astype(str) + "\nBrightness I-5"+ data['bright_ti5'].astype(str) + "\nFRP: "+ data['frp'].astype(str)

In [7]:
data['frp'].describe()

count    38508.000000
mean        13.487073
std         39.793591
min          0.100000
25%          1.800000
50%          3.900000
75%          9.900000
max       1142.500000
Name: frp, dtype: float64

In [16]:
frp_scaled = np.log(data['frp'])
frp_scaled += -2*frp_scaled.min()

In [17]:
pd.Series(frp_scaled).describe()

count    38508.000000
mean         6.139376
std          1.293760
min          2.302585
25%          5.192957
50%          5.966147
75%          6.897705
max         11.646144
Name: frp, dtype: float64

In [ ]:
###extract

In [3]:
rdata = data[["longitude","latitude","acq_date","confidence","bright_ti5"]]
rdata["intensity"] = 1.0
for i in rdata.index :
    if rdata["confidence"][i]==("high") :
        rdata["intensity"][i] = 100*rdata["bright_ti5"][i]
    elif rdata["confidence"][i]==("nominal") :
        rdata["intensity"][i] = 50*rdata["bright_ti5"][i]
    elif rdata["confidence"][i]==("low") :
        rdata["intensity"][i] = rdata["bright_ti5"][i]
rdata.head()

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aa

,longitude,latitude,acq_date,confidence,bright_ti5,intensity
0,151.47574,59.56768,2020-09-26,nominal,282.3,14115.0
1,151.52393,59.59822,2020-09-26,nominal,283.5,14175.0
2,151.52197,59.60613,2020-09-26,nominal,282.8,14140.0
3,151.51353,59.60234,2020-09-26,low,285.0,285.0
4,151.51282,59.61081,2020-09-26,nominal,284.6,14230.0


In [4]:
scaler = MinMaxScaler(feature_range=(0,5))
rdata["intensity"] = scaler.fit_transform(rdata[["intensity"]])
rdata["intensity"] = rdata["intensity"].round().astype(int)
rdata.head()

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/aaditya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,longitude,latitude,acq_date,confidence,bright_ti5,intensity
0,151.47574,59.56768,2020-09-26,nominal,282.3,2
1,151.52393,59.59822,2020-09-26,nominal,283.5,2
2,151.52197,59.60613,2020-09-26,nominal,282.8,2
3,151.51353,59.60234,2020-09-26,low,285.0,0
4,151.51282,59.61081,2020-09-26,nominal,284.6,2


In [ ]:
###Scatter_geo plot with slider

In [5]:
mark_values = {
    1 : "2020-09-26",
    2 : "2020-09-27",
    3 : "2020-09-28",
    4 : "2020-09-29",
    5 : "2020-09-30",
    6 : "2020-10-01",
    7 : "2020-10-02",
    8 : "2020-10-03"
}

In [6]:
app = dash.Dash()
app.layout = html.Div([
    
    html.Div([
        dcc.Graph(id="viirs")
    ]),
    
    html.Div([
        html.Label(["Choose a day to view data"],style={"text-align":"center","font-weight":"bold"}),
    ]),
        
        
    html.Div([
       dcc.Slider(
           id="slider",
           min = 1,
           max = 8,
           value = 1,
           marks = mark_values,
           step = None,
        ), 
    ],style={"width":"70%","position":"absolute","left":"5%"}),
        
])

In [7]:
@app.callback(
    Output('viirs','figure'),
    [Input('slider','value')],
)
def update_graph(day_choice):
    day_choice = mark_values[day_choice]
    trace = rdata.loc[rdata["acq_date"]==day_choice]
    days_scatter = px.scatter_geo(trace.head(1000),
                                  lat="latitude",
                                  lon="longitude",
                                  color="intensity",
                                  projection="natural earth",
                     )
    days_scatter.update_traces(textposition="top center")
    return(days_scatter)

In [8]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
